In [1]:
from open_clip import create_model_from_pretrained, get_tokenizer

model, preprocess = create_model_from_pretrained('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

/home/armand/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/armand/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/armand/miniconda3/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


open_clip_pytorch_model.bin:   0%|          | 0.00/784M [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

/home/armand/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

/home/armand/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/armand/miniconda3/lib/python3.10/site-packages/open_clip/factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_gl

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

In [29]:
import torch
from urllib.request import urlopen
from PIL import Image

dataset_url = 'https://huggingface.co/microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224/resolve/main/example_data/biomed_image_classification_example_data/'
test_imgs = [
    'squamous_cell_carcinoma_histopathology.jpeg',
    'H_and_E_histopathology.jpg',
    'bone_X-ray.jpg',
    'adenocarcinoma_histopathology.jpg',
    'covid_line_chart.png',
    'IHC_histopathology.jpg',
    'chest_X-ray.jpg',
    'brain_MRI.jpg',
    'pie_chart.png'
]

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()


context_length = 256

images = torch.stack([preprocess(Image.open(urlopen(dataset_url + img))) for img in test_imgs]).to(device)
texts = tokenizer(["" for im in images], context_length=context_length).to(device)
with torch.no_grad():
    image_features, text_features, logit_scale = model(images, texts)

print(image_features.shape)

torch.Size([9, 512])


In [11]:
with torch.no_grad():
    image_features, text_features, logit_scale = model(images, texts)
print(image_features[:,:3])

tensor([[-0.0279,  0.0556, -0.2120],
        [ 0.0046, -0.0682, -0.2127],
        [ 0.0295, -0.0657, -0.2510],
        [-0.0197,  0.0414, -0.1855],
        [ 0.0547,  0.0121, -0.0997],
        [ 0.0119, -0.0210, -0.2200],
        [-0.0434, -0.0100, -0.2650],
        [ 0.0150, -0.0132, -0.2424],
        [-0.0262, -0.0522, -0.2326]], device='cuda:0')


In [83]:
from fff.subject_model import BiomedClipModel

In [84]:
compiled_model = BiomedClipModel(model, tokenizer)

In [85]:
im_feat = compiled_model.encode(to_tensor(Image.open(urlopen(dataset_url + "H_and_E_histopathology.jpg"))).unsqueeze(0).to(device))

In [86]:
images = torch.stack([preprocess(Image.open(urlopen(dataset_url + "H_and_E_histopathology.jpg")))]).to(device)
texts = tokenizer([""], context_length=0).to(device)

im_feat_orig, _, _ = model(images, texts)

In [87]:
torch.mean((im_feat_orig - im_feat)**2)

tensor(1.5150e-08, device='cuda:0', grad_fn=<MeanBackward0>)

In [88]:
torch.save(compiled_model, "SubjectModels/saved_models/BiomedClip.pt")

In [79]:
print(im_feat.shape)

torch.Size([1, 512])


In [81]:
%ls

 BiomedModel.ipynb                     'FIF PythAE benchmark'/
'FFF Boltzmann Generator Evaluation'/  'Sine Experiments.ipynb'
 Fiber_Learning/                        SubjectModels/
 FiberModels/
